In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
import xgcm

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(10)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.0.145:50806 Dashboard: http://10.148.0.145:8787/status,Cluster Workers: 20 Cores: 80 Memory: 1000.00 GB


--- 

# For each hour

In [5]:
V = ['Eta']

out_dir = '/home1/scratch/xyu/mit_zarr/Eta/'  
Nt = 1

for v in V:

    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    it = np.arange(1512,2232).astype(int)*Nt # tmp
    it = np.arange(1512,1517).astype(int)*Nt # tmp

    assert it[-1]+Nt<time.size
    #
    p = 'C'
    vdrop = ['XC','YC','CS','SN','rA','Depth']
    if v is 'SSU':
        p = 'W'
        vdrop = ['dxC','dyG','rAw']         
    elif v is 'SSV':
        p = 'S'
        vdrop = ['dxG','dyC','rAs']        
        #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        #
        
    for i, t in enumerate(it):
        #
        file_out = out_dir+'%s_t%04d.zarr'%(v,i)
        dv = ds[v].isel(time=slice(t,t+Nt)) 
        dv = dv.drop(vdrop).to_dataset()
        dv.to_zarr(file_out, mode='w')     

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


_____
# For each face

In [6]:
V = ['Eta']
out_dir = '/home1/scratch/xyu/mit_zarr/'  
face_all = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] # some key faces

for face in face_all :
    
    for v in V:

    #
        data_dir = root_data_dir+v+'/'
        iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
        p = 'C'
        vdrop = ['XC','YC','CS','SN','rA','Depth']
        if v is 'SSU':
            p = 'W'
            vdrop = ['dxC','dyG','rAw']
        elif v is 'SSV':
            p = 'S'
            vdrop = ['dxG','dyC','rAs']
    #
        ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
        dv = ds[v].isel(time=slice(1512,2952),face=face)
        dv = dv.chunk({'time': 1,'i':4320,'j':4320})
        dv = dv.drop(vdrop).to_dataset()
        file_out = out_dir+'/%s_f%02d.zarr'%(v,face)
        %time dv.to_zarr(file_out, mode='w')                    
        dsize = getsize(file_out)
        print('face=%d - allready processed'%(face))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 17.5 s, sys: 992 ms, total: 18.5 s
Wall time: 1min 7s
face=0 - allready processed


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 15.2 s, sys: 792 ms, total: 16 s
Wall time: 36.4 s
face=1 - allready processed


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


...

CPU times: user 15 s, sys: 804 ms, total: 15.8 s
Wall time: 37 s
face=10 - allready processed


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 15.2 s, sys: 724 ms, total: 15.9 s
Wall time: 37.1 s
face=11 - allready processed


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


CPU times: user 15.3 s, sys: 912 ms, total: 16.2 s
Wall time: 38 s
face=12 - allready processed


In [7]:
cluster.scheduler.close()

<Future finished result=None>